In [260]:
from elasticsearch import Elasticsearch, helpers, exceptions
#from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime, timedelta
import time
#para request
#import requests
#from getpass import getpass
import math
import ipaddress # convertir IP
import parametros #parametros propios del desarrollo
from tld import get_tld #para manipulación de URLs
from tld.utils import update_tld_names
update_tld_names()

True

### Función para gestión de URLs

In [261]:
def maneja_url(url):
    try:
        res = get_tld(url, fix_protocol=True, as_object=True)
        # opciones de retorno: res,res.subdomain,res.domain,res.tld,res.fld
        return res.fld
    except:
        return url

Se realizan 3 lecturas al mismo indice crudo (paramtros.sonicwall_raw) con los siguientes criterios:
1. Traer los URL de las páginas:
* el criterio de filtrado es cuando el campo netflow.url-flow tiene datos
* de esta lectura se obtienen los datos netflow.url-time(fecha de visita) y netflow.url-name(URL visidata), de la cual solo se tomará el dominio.
* Este flujo cruza con el 2 y el 3.

2. Traer la IP del acces point desde donde se consulta la URL:
* netflow.url-flow-identify traiga algo (esta es la llave de cruce)
* el criterio de filtrado es cuando el campo netflow.url-flow-ip-origen tiene datos
* este flujo cruza con el 1 por medio del campo netflow.url-flow.

3. Categorias de las URL
* el criterio de filtrado es cuando el campo netflow.url-flow-categoria tiene datos.
* de esta lectura se obtiene el campo netflow.url-categoria-name2, el cual es la llave para cruzar con con el archivo "Mapeo-categorias-url.tsv"
* Esta lectura cruza con la lectura 1, usando el campo netflow.url-name.
* De la URL solo se debe tomar el dominio. Las jerarquías inferiores se descartan
* En cada lectura, si la categoría(netflow.url-categoria-name2) no se consigue en el archivo de mapeos, se agrega. Este proceso se hará inicialmente cargando el archivo al indice, después solo se usará el indice sonicwall_categorias
* este flujop cruza con el 1

#Definiendo fechas para rangos de consultas

In [208]:
now = datetime.now()
new_date = now - timedelta(days=6)
format_ES = "%Y.%m.%d"
ahora = str(now.strftime("%Y-%m-%dT%H:%M:%S"))
fecha_6 = str(new_date.strftime("%Y-%m-%dT%H:%M:%S"))
fecha_hoy = str(now.strftime(format_ES))

* Limite inferior del rango es la maxima fecha en el indice curado de sonicwall. En caso de no existir. Se calcula del indice crudo filebeat restandole 5 minutos a la maxima fecha en este.
* Limite superior del rango es la maxima fecha de actualización en el indice crudo filebeat. 
* Las ejecuciones se hacen sobre el indice crudo de la fecha actual. No se toman históricos.

### Conexión a Elastic Search

In [209]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=300, max_retries=1, retry_on_timeout=True
)

### Indice crudo filebeat

In [210]:
index_raw = parametros.sonicwall_raw

### Funcion para construcción de JSON ES

In [211]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### Fecha maxima de indice sonicwall

Este se usa para descartar de los indices crudos los registros menores a esa fecha

In [262]:
total_docs = 0
try:
    response = es.search(
        index=parametros.sonicwall_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "fecha_control", "format": "yyyy-MM-dd HH:mm:ss"}}
                }
             },
        size=total_docs
    )
#     elastic_docs = response["aggregations"]
    fecha_max = response["aggregations"]["max_date"]['value_as_string']
    h_inicio = response["aggregations"]["max_date"]['value_as_string']
    #h_fin = (datetime.strptime(h_inicio, '%Y-%m-%dT%H:%M:%S')+timedelta(hours=2)).strftime("%Y/%m/%dT%H:%M:%S")
except:
    fecha_max = (new_date).strftime("%Y-%m-%d") + ' 00:00:00'
    h_inicio = fecha_max
    
    #h_fin = (datetime.strptime(h_inicio, '%Y-%m-%dT%H:%M:%S') + timedelta(hours=1)).strftime("%Y-%m-%dT%H:%M:%S")
print("Fecha maxima en sonic curado:", fecha_max,"| Fecha inicio:", h_inicio)

Fecha maxima en sonic curado: 2021-11-10 15:30:01 | Fecha inicio: 2021-11-10 15:30:01


In [213]:
#h_inicio = "2021-10-14 00:00:00"

### Fecha maxima del indice crudo filebeat

In [214]:
total_docs = 0
try:
    response = es.search(
        index= index_raw,
        body={
                "aggs" : {
                   "max_date": {"max": {"field": "@timestamp", "format": "yyyy-MM-dd HH:mm:ss"}}
                },
            "query": {
                "bool": {
                  "filter": [
                
                  {
                    "exists": {
                      "field":"netflow.url-flow"
                    }
                  }
                  ]
                }
              }
             },
        size=total_docs
    )
    elastic_docs = response["aggregations"]
    fecha_max_crudo = response["aggregations"]["max_date"]['value_as_string']
    #h_fin = (datetime.strptime(h_inicio, '%Y-%m-%dT%H:%M:%S')+timedelta(hours=2)).strftime("%Y/%m/%dT%H:%M:%S")
    print("Fecha maxima en indice crudo:", fecha_max_crudo)
except:
    print("Error leyendo indice crudo filebeat")
    #exit()

Fecha maxima en indice crudo: 2021-11-10 16:00:27


* h_inicio viene del indice curado
* h_fin viene de el indice crudo
* En caso de que la fechas inicio y fin sean de dias diferentes, se igualan al actual. Se asume un rango de 10 minutos por corrida.
* Cuando el rango de fechas es mayor a 10 minutos se ajusta.

### 1. Traer los URL de las páginas

Se ajusta el maximo de resultados por consulta en el indice

In [215]:
es.indices.put_settings(index=index_raw,
                        body= {"index" : {
                                "max_result_window" : 500000
                        }})

{'acknowledged': True}

In [216]:
# def cargarJSON(elastic_docs):
#     return pd.DataFrame([x["_source"]['netflow'] for x in elastic_docs])

In [217]:
def trae_datos(desde, hasta, total_docs):
    response = es.search(
        index= index_raw,
        body={
              "_source": ["netflow.url*","@timestamp"],
              "query": {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "@timestamp": {
                            "gte": desde.replace(" ","T"),
                            "lt": hasta.replace(" ","T")
                        }
                      }
                    }
                  ,
                  {
                    "exists": {
                      "field":"netflow.url-flow"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    
    datos_api = pd.DataFrame([x["_source"]['netflow'] for x in elastic_docs])
    
    #datos_api = pd.DataFrame(elastic_docs,['url-time', 'url-name', 'url-flow', 'url-ip'])
    return datos_api

Se incrementa la fecha de inicio ya que esta es el limite inferior del rango anteriormente procesado

In [218]:
#h_fin = h_fin[0:15]+'0:00'
h_inicio = (datetime.strptime(h_inicio, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=30)).strftime("%Y-%m-%d %H:%M:%S")
h_fin = (datetime.strptime(h_inicio, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=30)).strftime("%Y-%m-%d %H:%M:%S")
if h_inicio.split(" ")[0]<h_fin.split(" ")[0]:
    h_inicio = h_fin[0:10]+' 00:00:01'
    h_fin = h_fin[0:10]+' 00:30:00'

In [219]:
#h_inicio='2021-06-30 09:00:00'
#h_fin='2021-06-30 09:10:00'


In [220]:
sonic_paginas = trae_datos(h_inicio, h_fin,100000)

In [221]:
if sonic_paginas is None or sonic_paginas.empty:
    while (sonic_paginas is None or sonic_paginas.empty) and ((datetime.strptime(h_inicio, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S")) < (datetime.strptime(fecha_max_crudo, '%Y-%m-%d %H:%M:%S').strftime("%Y-%m-%d %H:%M:%S"))):
        h_inicio = (datetime.strptime(h_inicio, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        h_fin = (datetime.strptime(h_fin, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=10)).strftime("%Y-%m-%d %H:%M:%S")
        sonic_paginas = trae_datos(h_inicio, h_fin,100000)
else:
    pass

In [222]:
#h_inicio='2021-06-30 09:00:00'
#h_fin='2021-06-30 09:10:00'
print(sonic_paginas)

        url-time                                    url-name    url-flow  \
0     1636560010                 exo.nel.measure.office.net/  2178239018   
1     1636560011                           v58.tiktokcdn.com  2178239700   
2     1636560010              us-east1-aws.api.snapchat.com/  2178238465   
3     1636560010               us-east1-aws.api.snapchat.com  2178238465   
4     1636560010                        clients4.google.com/  2178239549   
...          ...                                         ...         ...   
2408  1636560031                     181.49.90.145/empty.php  2163877163   
2409  1636560032                     181.49.90.145/empty.php  2163877170   
2410  1636560032                     181.49.90.145/empty.php  2163877172   
2411  1636560032                     181.49.90.145/empty.php  2163877174   
2412  1636560032  connectivitycheck.gstatic.com/generate_204  2163877178   

      url-ip  
0       27.0  
1        0.0  
2       58.0  
3        0.0  
4       29.0

In [223]:
sonic_paginas.drop_duplicates(subset=['url-flow','url-ip', 'url-time', 'url-name'],inplace=True)
sonic_paginas['url-dominio'] = sonic_paginas['url-name'].apply(maneja_url)
print(sonic_paginas)

        url-time                                    url-name    url-flow  \
0     1636560010                 exo.nel.measure.office.net/  2178239018   
1     1636560011                           v58.tiktokcdn.com  2178239700   
2     1636560010              us-east1-aws.api.snapchat.com/  2178238465   
3     1636560010               us-east1-aws.api.snapchat.com  2178238465   
4     1636560010                        clients4.google.com/  2178239549   
...          ...                                         ...         ...   
2408  1636560031                     181.49.90.145/empty.php  2163877163   
2409  1636560032                     181.49.90.145/empty.php  2163877170   
2410  1636560032                     181.49.90.145/empty.php  2163877172   
2411  1636560032                     181.49.90.145/empty.php  2163877174   
2412  1636560032  connectivitycheck.gstatic.com/generate_204  2163877178   

      url-ip              url-dominio  
0       27.0               office.net  
1      

In [203]:
#sonic_paginas.to_excel("sonic_pagina.xlsx")

### 2. Traer la IP del access point desde donde se consulta la URL

In [224]:
try:
    total_docs = 500000
    response = es.search(
        index= index_raw,
        body={
              "_source": ["netflow.url*","@timestamp"],
              "query": {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "@timestamp": {
                            "gte": h_inicio.replace(" ","T"),
                            "lt": h_fin.replace(" ","T")
                        }
                      }
                    }
                  ,
                  {
                    "exists": {
                      "field":"netflow.url-flow-identify"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]["netflow"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_ip = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
    
    datos_ip = pd.DataFrame([x["_source"]['netflow'] for x in elastic_docs])
    
except:
    print("Error leyendo indice crudo URL",response)
    exit()

In [225]:
datos_ip.drop_duplicates(subset=['url-ip-destino', 'url-ip-origen', 'url-flow-identify'],inplace=True)
datos_ip = datos_ip.rename(columns={'url-flow-identify' : 'url-flow'})

In [226]:
datos_ip

,url-ip-origen,url-ip-destino,url-flow
0,1.760743e+08,2.398794e+09,2178239591
2,1.760902e+08,3.357182e+09,2178221339
3,1.688280e+08,3.357182e+09,2178230427
4,1.687612e+08,3.039893e+09,2178204263
5,1.745385e+08,3.357182e+09,2178239844
...,...,...,...
34595,3.197126e+09,3.120665e+09,2201386518
34598,3.197126e+09,3.120665e+09,2201386521
34599,1.686257e+08,3.039911e+09,2194660069
34600,1.686257e+08,3.039911e+09,2194660071


In [72]:
#datos_ip.to_excel("datosip.xlsx")

### Combinando URL con Names

In [227]:
#datos_ip.to_excel("datos_ip.xlsx")
#datos_ip.to_excel("sonic_paginas.xlsx")
print(datos_ip)
print("///////")
print(sonic_paginas)
sonic_paginas = pd.merge(sonic_paginas,  datos_ip, on=['url-flow'], how='inner')

       url-ip-origen  url-ip-destino    url-flow
0       1.760743e+08    2.398794e+09  2178239591
2       1.760902e+08    3.357182e+09  2178221339
3       1.688280e+08    3.357182e+09  2178230427
4       1.687612e+08    3.039893e+09  2178204263
5       1.745385e+08    3.357182e+09  2178239844
...              ...             ...         ...
34595   3.197126e+09    3.120665e+09  2201386518
34598   3.197126e+09    3.120665e+09  2201386521
34599   1.686257e+08    3.039911e+09  2194660069
34600   1.686257e+08    3.039911e+09  2194660071
34601   1.686257e+08    3.039911e+09  2194660349

[18090 rows x 3 columns]
///////
        url-time                                    url-name    url-flow  \
0     1636560010                 exo.nel.measure.office.net/  2178239018   
1     1636560011                           v58.tiktokcdn.com  2178239700   
2     1636560010              us-east1-aws.api.snapchat.com/  2178238465   
3     1636560010               us-east1-aws.api.snapchat.com  2178238465  

In [228]:
sonic_paginas

,url-time,url-name,url-flow,url-ip,url-dominio,url-ip-origen,url-ip-destino
0,1636560010,exo.nel.measure.office.net/,2178239018,27.0,office.net,176218674.0,1.611129e+09
1,1636560010,exo.nel.measure.office.net,2178239018,0.0,office.net,176218674.0,1.611129e+09
2,1636560011,v58.tiktokcdn.com,2178239700,0.0,tiktokcdn.com,168599111.0,1.546031e+09
3,1636560010,us-east1-aws.api.snapchat.com/,2178238465,58.0,snapchat.com,175053069.0,5.626113e+07
4,1636560010,us-east1-aws.api.snapchat.com,2178238465,0.0,snapchat.com,175053069.0,5.626113e+07
...,...,...,...,...,...,...,...
1975,1636560031,181.49.90.144/empty.php,2158448928,64.0,181.49.90.144/empty.php,170672378.0,3.039911e+09
1976,1636560031,bidder.criteo.com/,2163877162,54.0,criteo.com,169623866.0,1.249343e+09
1977,1636560031,181.49.90.145/getIP.php,2163877163,64.0,181.49.90.145/getIP.php,169623770.0,3.039911e+09
1978,1636560031,181.49.90.145/empty.php,2163877163,64.0,181.49.90.145/empty.php,169623770.0,3.039911e+09


In [806]:
#sonic_paginas.to_excel("paginas.xlsx")

### 3. Leyendo indice de categorías

* Se lee indice categorias, si no retorna nada se lee el plano separado por tabulador.
* La información leída del archivo es agregada al indice de categorías. En lecturas sucesivas ya estaría disponible el indice.

In [229]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.sonicwall_categorias,
        body={
                "_source": ["ID","Categoria"]
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        source_data = doc["_source"]
        for key, val in source_data.items():
            try:
                fields[key] = np.append(fields[key], val)
            except KeyError:
                fields[key] = np.array([val])

    datos_categorias = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))
except:
    datos_categorias = pd.read_csv('/home/desarrollo/gh-mintic-backend/archivos/Mapeo-categorias-url.tsv', sep='\t', encoding = 'utf-8')
    datos_categorias.dropna(inplace=True)
    use_these_keys = ['ID', 'Categoria','@timestamp']
    timestamp = datetime.now()
    datos_categorias['@timestamp'] = timestamp.isoformat()
    def doc_generator(df):
        df_iter = df.iterrows()
        for index, document in df_iter:
            yield {
                    "_index": sonicwall_categorias, 
                    "_id": f"{document['ID']}",
                    "_source": filterKeys(document),
                }
    salida = helpers.bulk(es, doc_generator(datos_categorias))
    print("Fecha: ", now,"- Datos nuevos en indice sonic-categorias:",salida[0])
    #datos_sonic = datos_sonic.drop(aux_recurrencia[(aux_recurrencia['contador'] < 2)].index)


### Se lee del indice crudo las Categorias

In [230]:
try:
    total_docs = 500000
    response = es.search(
        index= index_raw,
        body={
              "_source": ["netflow.url*","@timestamp"],
              "query": {
                "bool": {
                  "filter": [
                    {
                      "range": {
                        "@timestamp": {
                            "gte": h_inicio.replace(" ","T"),
                            "lt": h_fin.replace(" ","T")
                        }
                      }
                    }
                  ,
                  {
                    "exists": {
                      "field":"netflow.url-categoria-name2"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]["netflow"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])
#     sonic_categorias = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ]))

    sonic_categorias = pd.DataFrame([x["_source"]['netflow'] for x in elastic_docs])
    
except:
    print("Error leyendo indice crudo Categorías")
    #exit()

In [77]:
#prueba = json.loads(elastic_docs)
#prueba
#df = pd.json_normalize(prueba['results'])

* Se  identifica el dominio de cada URL
* Se descartam duplicados por dominio y categoría
* Se renombre ID para cruce

In [231]:
sonic_categorias.drop_duplicates(subset=['url-name','url-categoria-name2'],inplace=True)
sonic_categorias = sonic_categorias.rename(columns={'url-categoria-name2' : 'ID'})
print(sonic_categorias)

                                               url-name  ID  \
0                                           dns.google/  27   
1                   im16-normal-c-useast1a.tiktokv.com/  58   
2                            partners-ir.peel-prod.com/  15   
3                                       www.google.com/  29   
4                                www.google.com/gen_204  29   
...                                                 ...  ..   
2350               mon16-normal-c-useast1a.tiktokv.com/  58   
2351                          callback.ad.snaptube.app/  56   
2360                                   lm.facebook.com/  58   
2366  ocsp.digicert.com/MFEwTzBNMEswSTAJBgUrDgMCGgUA...  27   
2371            assets.nvidiagrid.net/titles/nvidiatech  15   

      url-categoria-name5  url-categoria-name3  url-categoria-name4  
0                       0                    0                    0  
1                       0                    0                    0  
2                       0        

### Se  combina las categorias que vienen del indice con las existentes en el indice de categorías

In [232]:
sonic_categorias = pd.merge(sonic_categorias[['url-name','ID']],  datos_categorias, on='ID', how='outer')

In [233]:
#sonic_categorias.to_excel("categorias.xlsx")
#sonic_paginas.to_excel("sonic_paginas.xlsx")
print(sonic_paginas)

        url-time                                    url-name    url-flow  \
0     1636560010                 exo.nel.measure.office.net/  2178239018   
1     1636560010                  exo.nel.measure.office.net  2178239018   
2     1636560011                           v58.tiktokcdn.com  2178239700   
3     1636560010              us-east1-aws.api.snapchat.com/  2178238465   
4     1636560010               us-east1-aws.api.snapchat.com  2178238465   
...          ...                                         ...         ...   
1975  1636560031                     181.49.90.144/empty.php  2158448928   
1976  1636560031                          bidder.criteo.com/  2163877162   
1977  1636560031                     181.49.90.145/getIP.php  2163877163   
1978  1636560031                     181.49.90.145/empty.php  2163877163   
1979  1636560032  connectivitycheck.gstatic.com/generate_204  2163877178   

      url-ip              url-dominio  url-ip-origen  url-ip-destino  
0       27.0    

### Se combina categorías con indice principal

In [234]:
sonic_paginas = pd.merge(sonic_paginas,  sonic_categorias, on='url-name', how='inner')
print(sonic_paginas)

        url-time                        url-name    url-flow  url-ip  \
0     1636560010     exo.nel.measure.office.net/  2178239018    27.0   
1     1636560010  us-east1-aws.api.snapchat.com/  2178238465    58.0   
2     1636560010  us-east1-aws.api.snapchat.com/  2178238022    58.0   
3     1636560010  us-east1-aws.api.snapchat.com/  2178238613    58.0   
4     1636560010  us-east1-aws.api.snapchat.com/  2178238614    58.0   
...          ...                             ...         ...     ...   
1002  1636560020                     t.teads.tv/  2178257202    15.0   
1003  1636560019            api.ad.snaptube.app/  2178255823    56.0   
1004  1636560030         181.49.90.144/getIP.php  2158448923    64.0   
1005  1636560029             pebed.dm-event.net/  2163877150    15.0   
1006  1636560031         181.49.90.145/getIP.php  2163877163    64.0   

                  url-dominio  url-ip-origen  url-ip-destino  ID  \
0                  office.net    176218674.0    1.611129e+09  27   

* Si la categoría no existe en el indice de categorías, se agrega ascociando el dominio como nombre de la categoría
* Se genera un nuevo dataframe con los valores que no existan en indice para luego insertarlos
* Para incorporar en flujo principal, se sustituye el valor Categorías nulas por el valor del url-dominio

In [235]:
sonic_categorias['url-dominio'] = sonic_categorias['url-name'].apply(maneja_url)
sonic_categorias['Categoria'].fillna(sonic_categorias['url-dominio'], inplace=True)
categoria_update = sonic_categorias[(sonic_categorias['Categoria'].isnull())][['url-dominio','ID']]

### Convirtiendo fechas e IPs

In [236]:
sonic_paginas['url-time'] = [time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)) for x in sonic_paginas['url-time']]

In [237]:
sonic_paginas.fillna({'url-ip-destino':0, 'url-ip-origen':0},inplace=True)
#sonic_paginas.fillna({'url-ip-origen':0},inplace=True)
sonic_paginas = sonic_paginas.astype({"url-ip-destino": int, "url-ip-origen":int})

In [238]:
sonic_paginas['url-ip-destino'] = [str(ipaddress.ip_address(x)) for x in sonic_paginas['url-ip-destino']]
sonic_paginas['url-ip-origen'] = [str(ipaddress.ip_address(x)) for x in sonic_paginas['url-ip-origen']]

In [239]:
sonic_paginas.drop_duplicates(inplace=True)

Se extrae solo los 3 primeros octetos de la IP origen

In [240]:
print(sonic_paginas['url-ip-origen'])

0       10.128.226.50
1        10.111.25.13
2        10.111.25.13
3        10.111.25.13
4        10.111.25.13
            ...      
1002    10.22.221.103
1003    10.112.124.39
1004     10.44.64.250
1005      10.28.5.214
1006     10.28.64.218
Name: url-ip-origen, Length: 1007, dtype: object


In [241]:
if (len(sonic_paginas) > 0):
    sonic_paginas['segmento'] = sonic_paginas['url-ip-origen'].str.split(".", n = 1, expand = True)[0] +'.'+ sonic_paginas['url-ip-origen'].str.split(".", n = 2, expand = True)[1] +'.'+ sonic_paginas['url-ip-origen'].str.split(".", n = 3, expand = True)[2]

In [243]:
  sonic_paginas

,url-time,url-name,url-flow,url-ip,url-dominio,url-ip-origen,url-ip-destino,ID,Categoria,segmento
0,2021-11-10 11:00:10,exo.nel.measure.office.net/,2178239018,27.0,office.net,10.128.226.50,96.7.224.27,27,Information Technology/Computer,10.128.226
1,2021-11-10 11:00:10,us-east1-aws.api.snapchat.com/,2178238465,58.0,snapchat.com,10.111.25.13,3.90.122.13,58,Social Networking,10.111.25
2,2021-11-10 11:00:10,us-east1-aws.api.snapchat.com/,2178238022,58.0,snapchat.com,10.111.25.13,3.90.122.13,58,Social Networking,10.111.25
3,2021-11-10 11:00:10,us-east1-aws.api.snapchat.com/,2178238613,58.0,snapchat.com,10.111.25.13,3.90.122.13,58,Social Networking,10.111.25
4,2021-11-10 11:00:10,us-east1-aws.api.snapchat.com/,2178238614,58.0,snapchat.com,10.111.25.13,3.90.122.13,58,Social Networking,10.111.25
...,...,...,...,...,...,...,...,...,...,...
1002,2021-11-10 11:00:20,t.teads.tv/,2178257202,15.0,teads.tv,10.22.221.103,23.207.53.46,15,Business and Economy,10.22.221
1003,2021-11-10 11:00:19,api.ad.snaptube.app/,2178255823,56.0,snaptube.app,10.112.124.39,18.235.117.75,56,Other,10.112.124
1004,2021-11-10 11:00:30,181.49.90.144/getIP.php,2158448923,64.0,181.49.90.144/getIP.php,10.44.64.250,181.49.90.144,64,Not Rated,10.44.64
1005,2021-11-10 11:00:29,pebed.dm-event.net/,2163877150,15.0,dm-event.net,10.28.5.214,188.65.124.59,15,Business and Economy,10.28.5


### Se lee semilla

In [244]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','IP PROD_1','IP PROD_2']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
#     fields = {}
#     for num, doc in enumerate(elastic_docs):
#         source_data = doc["_source"]
#         for key, val in source_data.items():
#             try:
#                 fields[key] = np.append(fields[key], val)
#             except KeyError:
#                 fields[key] = np.array([val])

#     datos_semilla = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in fields.items() ])) #pd.DataFrame(fields)

    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    exit()

Se extrae solo los 3 primeros octetos del IP OUTDOOR1 en semilla para cruzar con Sonic

In [245]:
df1=datos_semilla
df1['segmento'] = datos_semilla['IP PROD_1'].str.split(".", n = 1, expand = True)[0] +'.'+ datos_semilla['IP PROD_1'].str.split(".", n = 2, expand = True)[1] +'.'+ datos_semilla['IP PROD_1'].str.split(".", n = 3, expand = True)[2]
#df1

In [246]:
df2 = datos_semilla
df2['segmento'] = datos_semilla['IP PROD_2'].str.split(".", n = 1, expand = True)[0] +'.'+ datos_semilla['IP PROD_2'].str.split(".", n = 2, expand = True)[1] +'.'+ datos_semilla['IP PROD_2'].str.split(".", n = 3, expand = True)[2]
#df2

In [247]:
# df2.index=df2.index+datos_semilla.shape[0]

In [248]:
df3=pd.DataFrame(pd.concat([df1,df2]))

In [249]:
if (len(sonic_paginas) > 0):
    sonic_paginas = pd.merge(sonic_paginas, df3, on = 'segmento', how='inner')

In [250]:
#sonic_paginas['site_id']

In [251]:
# sonic_paginas = pd.merge(sonic_paginas, datos_semilla, on = 'segmento', how='inner')

### Se totaliza para rango de fecha control

* Se generan dos dataframe los cuales se insertan de forma independiente en el mismo indice
* la agrupación de sitios mas visitados, se hacer por el dominio para dar mejor visualización al usuario

In [252]:
#sonic_paginas['fecha_control'] = sonic_paginas['url-time'].str[0:15]+'0:00'
if (len(sonic_paginas) > 0):
    sonic_paginas['fecha_control'] = h_inicio

In [253]:
total_categoria = pd.DataFrame()
if (len(sonic_paginas) > 0):
    total_categoria = sonic_paginas[['url-time','site_id','fecha_control','Categoria']].groupby(['site_id','fecha_control','Categoria']).agg(['count']).reset_index()
    total_categoria.columns = total_categoria.columns.droplevel(1)
    total_categoria.rename(columns={'url-time': 'total.categoria'}, inplace=True)

In [255]:
total_dominio = pd.DataFrame()
if (len(sonic_paginas) > 0):
    total_dominio = sonic_paginas[['url-time','site_id','fecha_control','Categoria','url-dominio']].groupby(['site_id','fecha_control','Categoria','url-dominio']).agg(['count']).reset_index()
    total_dominio.columns = total_dominio.columns.droplevel(1)
    total_dominio.rename(columns={'url-time': 'total.dominio'}, inplace=True)

### Se escribe la información al indice sonic curado

In [256]:
use_these_keys = ['site_id', 'fecha_control', 'Categoria', 'total.categoria','@timestamp']

total_categoria['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'sonic_wall_ingtest', 
                "_id": f"{str(document['site_id'])+'-'+ str(document['fecha_control'])+'-'+ str(document['Categoria'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(total_categoria))
print("Fecha: ", now,"- Datos CATEGORIAS en sonic-curado:",salida[0])

Fecha:  2021-11-10 11:00:22.737404 - Datos CATEGORIAS en sonic-curado: 495


In [257]:
use_these_keys = ['site_id', 'fecha_control', 'url-dominio','Categoria', 'total.dominio','@timestamp']

total_dominio['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'sonic_wall_ingtest', 
                "_id": f"{str(document['site_id'])+'-'+ str(document['fecha_control'])+'-'+str(document['Categoria'])+'-'+ str(document['url-dominio'])}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(total_dominio))
print("Fecha: ", now,"- Datos DOMINIOS en sonic-curado:",salida[0])

Fecha:  2021-11-10 11:00:22.737404 - Datos DOMINIOS en sonic-curado: 552


### Se actualiza indice categorías

In [258]:
categoria_update = categoria_update.rename(columns={'url-dominio' : 'Categoria'})

In [259]:
use_these_keys = ['ID', 'Categoria','@timestamp']
categoria_update['@timestamp'] = now.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'sonic_wall_catingtest', 
                "_id": f"{document['ID']}",
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(categoria_update))
print("Fecha: ", now,"- Insertados en indice sonic-categorias:",salida[0])

Fecha:  2021-11-10 11:00:22.737404 - Insertados en indice sonic-categorias: 0
